In [2]:
import geopandas as gpd
from geopandas import GeoDataFrame
import os
import zipfile
import pandas as pd
import glob
import zipfile
from pandas import DataFrame
layer_crs= 'epsg:2039'
project_folder = os.path.dirname(os.getcwd())
from shapely.geometry import LineString

In [3]:
# The network to work with
res_path = 'output/streets_elements'
clean_network = gpd.read_file(r'{}/streets_elements.shp'.format(res_path))
col_to_leave = ['oidrechov','length','geometry']


In [4]:
key_fields = 'oidrechov'

In [5]:
# Code for building  'output/streets_elements/elements/buildings_business\\businesses.shp']
file = 'output/streets_elements/elements/buildings_business\\buildings.shp'
clean_network_temp =clean_network[col_to_leave]
clean_network_temp[['count','sum']] = 0

# clean_network_temp['count'], sjoin,name,path= calculate_count(file)
name = file.split("\\")[-1].split('.')[0]
field = 'govasimple'

print(name)

objects_file = gpd.read_file(file).drop(columns=['index'])
objects_file = objects_file[objects_file[field]>3]


C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


buildings


In [6]:
# prepare data
# Buffer around each building
path = 'output/streets_elements/detailed_folder/buildings_business/{}'.format(name)
buffers_df = objects_file.buffer(distance=50,join_style=2)

GeoDataFrame(geometry=buffers_df,crs=layer_crs).reset_index().to_file(f'{path}/buffer.shp')
# reset_index in order to save the index file when presented on ArcGIS
objects_file.reset_index().to_file((f'{path}/filter_height.shp'))

# find for each polygon all the polylines intersected with him. leave only those with at least one intersection
s_join = GeoDataFrame(geometry=buffers_df,crs=layer_crs).sjoin(clean_network_temp,how='inner',lsuffix='')
# reset_index in order to save the index file when presented on ArcGIS, inplace=True since it also used later
s_join.reset_index(inplace=True)
s_join.to_file((f'{path}/s_join.shp'))
# Find the centroid of each building
centroid = objects_file.centroid
GeoDataFrame(centroid.reset_index().rename(columns ={0:'geometry'}),crs=layer_crs).to_file(f'{path}/centroid.shp')

C:\Users\Achituv\AppData\Local\Temp\ipykernel_5596\3626553475.py:14: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  s_join.to_file((f'{path}/s_join.shp'))


In [ ]:
# Create new polylines between the buildings to the streets around
new_lines ={'building':[],'road':[],'geometry':[]}
def create_new_polyline(row):
    road_index = row['index_right']
    building_index = row['index']
    centroid_0 = centroid.loc[building_index]
    line = clean_network_temp.loc[road_index]['geometry']
    new_point = list(line.interpolate(line.project(centroid_0)).coords)[0]
    if  (new_point[0]==line.xy[0][0] and new_point[1]==line.xy[1][0]) or (new_point[0]==line.xy[0][-1] and new_point[1]==line.xy[1][-1]):
        return
    neline = LineString([list(centroid_0.coords)[0], new_point])
    new_lines['geometry'].append(neline)
    new_lines['road'].append(road_index)
    new_lines['building'].append(building_index)
s_join.apply(create_new_polyline,axis=1)
new_lines_df= GeoDataFrame(new_lines,crs=layer_crs)
new_lines_df.to_file(f'{path}/new_lines.shp')

In [14]:
new_lines_buildings_0 = new_lines_df.sjoin(objects_file,how='left',lsuffix='')
no_intersect = []
for group in new_lines_buildings_0.groupby(['building','road']):
    if len(group[1])==1:
        no_intersect.append(group[1])
no_inter_building = pd.concat(no_intersect)

In [17]:
no_inter_building.drop(columns='index_right',inplace=True)
s_join2 =no_inter_building.sjoin( GeoDataFrame(geometry=clean_network_temp.buffer(distance=1,join_style=2),crs=layer_crs),how='left',lsuffix='')
s_join2.to_file(f'{path}/s_join2.shp')

In [39]:
s_join2['wkt']= s_join2.apply(lambda x: x['geometry'].wkt,axis=1)
s_join2


,building,road,geometry,govasimple,name,index_right,wkt
9,2,6319,"LINESTRING (185244.253 669013.307, 185247.286 ...",6.69,export_20220818_150110,6319,LINESTRING (185244.2527250059 669013.307291207...
0,2,7077,"LINESTRING (185244.253 669013.307, 185231.993 ...",6.69,export_20220818_150110,7077,LINESTRING (185244.2527250059 669013.307291207...
10,3,6319,"LINESTRING (185218.875 669045.615, 185217.337 ...",6.26,export_20220818_150110,6319,LINESTRING (185218.87507689887 669045.61514012...
1,3,7077,"LINESTRING (185218.875 669045.615, 185200.249 ...",6.26,export_20220818_150110,7077,LINESTRING (185218.87507689887 669045.61514012...
1,3,7077,"LINESTRING (185218.875 669045.615, 185200.249 ...",6.26,export_20220818_150110,6319,LINESTRING (185218.87507689887 669045.61514012...
...,...,...,...,...,...,...,...
117473,47237,6417,"LINESTRING (177309.377 660205.841, 177341.862 ...",9.79,export_20220818_150955,6417,LINESTRING (177309.37698045632 660205.84050648...
117351,47237,8436,"LINESTRING (177309.377 660205.841, 177311.996 ...",9.79,export_20220818_150955,8436,LINESTRING (177309.37698045632 660205.84050648...
117135,47240,2548,"LINESTRING (175928.959 660158.345, 175919.031 ...",6.52,export_20220818_150955,2548,LINESTRING (175928.95865433384 660158.34465438...
117142,47240,2549,"LINESTRING (175928.959 660158.345, 175956.171 ...",6.52,export_20220818_150955,2549,LINESTRING (175928.95865433384 660158.34465438...


In [40]:
no_intersect2 = []
for group in s_join2.groupby('wkt'):
    if len(group[1])==1:
        no_intersect2.append(group[1])
no_inter_building2 = pd.concat(no_intersect2)
no_inter_building2.to_file((f'{path}/no_intersect_line.shp'))

C:\Users\Achituv\AppData\Local\Temp\ipykernel_5596\2698935623.py:2: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group in s_join2.groupby(['wkt']):
C:\Users\Achituv\AppData\Local\Temp\ipykernel_5596\2698935623.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  no_inter_building2.to_file((f'{path}/no_intersect_line.shp'))


In [42]:
# add number of building and avarage height
dic_count={}
dic_height ={}
for group in no_inter_building2.groupby(['building','road']):
    if len(group[1])==1:
        road = group[0][1]
        if road in dic_count:
            dic_count[road]+=1
            dic_height[road]+= group[1]['govasimple'].values[0]
        else:
            dic_count[road]=1
            dic_height[road]= group[1]['govasimple'].values[0]
    else:
        print(f'error-{group[0]}')
clean_network_temp['count'] =dic_count
clean_network_temp['sum'] = dic_height
clean_network_temp['avg']= clean_network_temp['sum']/clean_network_temp['count']
clean_network_temp[['count','sum']].fillna(0,inplace=True)
clean_network_temp.to_file('{}/streets_elements_{}.shp'.format(path,name))
print('finish')

C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Achituv\AppData\Local\Temp\ipykernel_5596\2955999986.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the c

finish


In [46]:
clean_network[name] = clean_network_temp['avg']
clean_network.to_file('output/streets_elements/streets_elements_temp.shp')

In [45]:
clean_network_temp

,oidrechov,length,geometry,count,sum,avg
0,1.0,32.807598,"LINESTRING (184322.705 668574.483, 184351.736 ...",3.0,48.11,16.036667
1,3.0,63.288914,"LINESTRING (179282.603 666005.717, 179343.283 ...",4.0,103.72,25.930000
2,4.0,25.981647,"LINESTRING (180952.197 664967.670, 180964.041 ...",3.0,44.00,14.666667
3,5.0,80.268853,"LINESTRING (180668.486 662875.092, 180748.697 ...",13.0,100.79,7.753077
4,6.0,80.152149,"LINESTRING (182532.174 661455.967, 182514.127 ...",5.0,44.00,8.800000
...,...,...,...,...,...,...
8746,9650.0,92.387138,"LINESTRING (180252.127 670097.359, 180342.969 ...",NaN,NaN,NaN
8747,9651.0,55.993258,"LINESTRING (180204.072 670123.951, 180220.846 ...",NaN,NaN,NaN
8748,9658.0,1416.463909,"LINESTRING (180765.260 670000.467, 180830.127 ...",5.0,71.78,14.356000
8749,9659.0,720.172594,"LINESTRING (181058.769 671381.112, 181091.908 ...",NaN,NaN,NaN


In [162]:

clean_network.to_file('output/streets_elements/streets_elements_temp.shp')


finish


In [163]:
# Code for building  'output/streets_elements/elements/buildings_business\\businesses.shp']
file = 'output/streets_elements/elements/buildings_business\\businesses.shp'
file_buildings = r'output\streets_elements\detailed_folder\buildings_business\buildings\streets_elements_buildings.shp'
clean_network_temp = clean_network[col_to_leave]
clean_network_temp[['count','count_b', 'ratio','ratio_b']] = 0

# clean_network_temp['count'], sjoin,name,path= calculate_count(file)
name = file.split("\\")[-1].split('.')[0]

print(name)

objects_file = gpd.read_file(file)
# Find the nearest street segment for each object
sjoin = objects_file.sjoin_nearest(clean_network_temp, distance_col='dis', max_distance=50)
path = 'output/streets_elements/detailed_folder/buildings_business/{}'.format(name)
os.makedirs(path, exist_ok=True)
sjoin[['geometry','oidrechov','length','dis','count','count_b', 'ratio','ratio_b']].to_file('{}/sjoin.shp'.format(path))
file = 'output/streets_elements/elements/buildings_business\\{}.shp'.format(name)
clean_network_temp = clean_network[col_to_leave]


# Aggregate objects that share the same closet street
dissolve = DataFrame(sjoin).groupby(by=key_fields).count()
clean_network_temp.set_index(key_fields, inplace=True)
clean_network_temp['count'] = dissolve['length']
building_network =  gpd.read_file(file_buildings).set_index(key_fields)
clean_network_temp['ratio'] = clean_network_temp['count'] / building_network['length']
clean_network_temp['ratio_b'] = clean_network_temp['count'] / building_network['count']
clean_network_temp['count_b'] = building_network['count']

clean_network_temp.to_file('{}/streets_elements_{}.shp'.format(path, name))
print('finish')

C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

businesses


C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

finish


In [ ]:

# Find the nearest street segment for each object
# sjoin= objects_file.sjoin_nearest(clean_network_temp,distance_col='dis', max_distance=50)

# os.makedirs(path,exist_ok = True)
# sjoin.dropna(subset=[field],inplace=True)
# sjoin.to_file('{}/sjoin.shp'.format(path))
# file = 'output/streets_elements/elements/buildings_business\\buildings.shp'
# clean_network_temp =clean_network[col_to_leave]
#
#
# # Aggregate objects that share the same closet street
# dissolve = DataFrame(sjoin).groupby(by = key_fields).count()
# clean_network_temp.set_index(key_fields, inplace=True)
# clean_network_temp['count'] =dissolve['length']
#
#
#
# s_col = sjoin[[field,key_fields]]
# dissolve = s_col.groupby(by = key_fields).mean()
#
# clean_network_temp['avg'] = dissolve[field]
# clean_network.set_index(key_fields, inplace=True)
# clean_network[name] = clean_network_temp['avg']
# clean_network.reset_index(inplace=True)
# clean_network_temp.to_file('{}/streets_elements_{}.shp'.format(path,name))
# clean_network.to_file('output/streets_elements/streets_elements.shp')
# print('finish')